In [1]:
# import the bigquery
from google.cloud import bigquery

In [2]:
# initializing the client object
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [4]:
# asking client to take the reference
dataset_ref = client.dataset('hacker_news', project = 'bigquery-public-data')

dataset = client.get_dataset(dataset_ref)

In [8]:
tables = list(client.list_tables(dataset))

for i in tables:
    print(i.table_id)

comments
full
full_201510
stories


In [9]:
# wee need only comments table

table_ref = dataset_ref.table('comments')
table = client.get_table(table_ref)

In [10]:
table.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', 'Unique comment ID', ()),
 SchemaField('by', 'STRING', 'NULLABLE', 'Username of commenter', ()),
 SchemaField('author', 'STRING', 'NULLABLE', 'Username of author', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE', 'Human readable time in UTC (format: YYYY-MM-DD hh:mm:ss)', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Comment text', ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ())]

In [12]:
client.list_rows(table, max_results = 5).to_dataframe()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


Let's use the table to see which comments generated the most replies. Since:

* the parent column indicates the comment that was replied to, and
* the id column has the unique ID used to identify each comment,

we can GROUP BY the parent column and COUNT() the id column in order to figure out the number of comments that were made as responses to a specific comment. (This might not make sense immediately -- take your time here to ensure that everything is clear!)

Furthermore, since we're only interested in popular comments, we'll look at comments with more than ten replies. So, we'll only return groups HAVING more than ten ID's.

In [13]:
query = """ select parent, count(id) from `bigquery-public-data.hacker_news.comments` 
            group by parent
            having count(id) > 10   
         """

In [14]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)

# sending query to the database

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config = safe_config)


popular_comments = query_job.to_dataframe()

In [24]:
popular_comments.sort_values(by = ['f0_'], ascending = False).head()

,parent,f0_
4068,363,1311
2344,9812245,902
244,9996333,850
2308,9303396,785
2343,10152809,733


In [25]:
query_improved = """
                 SELECT parent, COUNT(1) AS NumPosts
                 FROM `bigquery-public-data.hacker_news.comments`
                 GROUP BY parent
                 HAVING COUNT(1) > 10
                 """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_improved, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
improved_df = query_job.to_dataframe()

# Print the first five rows of the DataFrame
improved_df.head()

,parent,NumPosts
0,6126926,49
1,780569,40
2,2772387,104
3,6930109,39
4,1820561,46


In [27]:
improved_df.sort_values(by = ['NumPosts'], ascending = False).head(10)

,parent,NumPosts
4634,363,1311
2141,9812245,902
49,9996333,850
2426,9303396,785
2292,10152809,733
2879,9471287,697
2321,10311580,645
2763,9639001,635
4403,9127232,529
972,8980047,526


## Note on using GROUP BY

Note that because it tells SQL how to apply aggregate functions (like COUNT()), it doesn't make sense to use GROUP BY without an aggregate function. Similarly, if you have any GROUP BY clause, then all variables must be passed to either a

* GROUP BY command, or
* an aggregation function.

Consider the query below:

In [ ]:
query_good = """
             SELECT parent, COUNT(id)
             FROM `bigquery-public-data.hacker_news.comments`
             GROUP BY parent
             """

Note that there are two variables: parent and id.

* parent was passed to a GROUP BY command (in GROUP BY parent), and
* id was passed to an aggregate function (in COUNT(id)).

And this query won't work, because the author column isn't passed to an aggregate function or a GROUP BY clause:

In [ ]:
query_bad = """
            SELECT author, parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY parent
            """

If make this error, you'll get the error message SELECT list expression references column (column's name) which is neither grouped nor aggregated at.